<a href="https://colab.research.google.com/github/jaunty0216/ForSophomoreCourse/blob/main/%E7%AC%AC%E5%9B%9B%E9%80%B1_%E9%82%8F%E8%BC%AF%E6%A2%9D%E4%BB%B6%E6%8E%A7%E5%88%B6AI_%E5%92%96%E5%93%A9%E9%A3%AF%E6%8B%8C%E8%88%87%E4%B8%8D%E6%8B%8C%E7%9A%84%E7%B5%82%E6%A5%B5%E8%BE%AF%E8%AB%96%E8%B3%BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 目標
這次的目標：**用邏輯條件控制gemini的AI**

---



# 初始化
**老樣子：**


1.   引入函式庫
2.   引用API
3.   選取模型



In [16]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

import google.generativeai as genai
genai.configure(api_key=GOOGLE_API_KEY)
model_1 = genai.GenerativeModel("gemini-1.5-flash")
model_2 = genai.GenerativeModel("gemini-1.5-flash")

# 準備紀錄辯論的資料工具
將資料記錄到**google sheet**裡面


1.   先登入帳號
2.   連結試算表
3.   紀錄資料(在辯論過程中)



In [76]:
# 登入，驗證帳號
from google.colab import auth

try:
  auth.authenticate_user()
  print("successfully accessed!")
except:
  print("error")

# 載入google sheet與相關的函式庫
from google.auth import default
import gspread

creds ,_ = default()
gc = gspread.authorize(creds)

# 載入google sheet
try:
  ProgrammingWeek4 = gc.open_by_url("https://docs.google.com/spreadsheets/d/161euJffjihgIKrZRsuhH6uJum3igq4cJ5kyqXmPup2Q/edit?usp=sharing")
  RecordingSheet = ProgrammingWeek4.get_worksheet(0)
  print("successfully accessed the worksheet!")
except:
  print("error")

successfully accessed!
successfully accessed the worksheet!


# 設定辯論局
1.   我想要讓兩個gemini的AI互相辯論，然後總共只有7回合
2.   使用者可以輸入辯論的主題
3.   這邊我想讓他們討論：咖哩飯拌與不拌(whether to stir curry with rice while having curry as a meal.)



In [75]:
def LetsGoDebating(topic):

  # 初始化
  PromptForFirstOrator = f"Please debate about {topic} with SecondOrator, you are supporting to the topic."
  PromptForSecondOrator = f"According to the content from FirstOrator, against his opinion and the topic."
  FirstOrator = model_1.generate_content(PromptForFirstOrator)
  SecondOrator = model_2.generate_content(PromptForSecondOrator)
  RoundIteration = 7
  current_row = 1

  # 把辯論資料寫到目標的google sheet
  from io import StringIO
  from gspread_dataframe import set_with_dataframe
  import pandas as pd

  #AssignedSheet = ProgrammingWeek4.get_worksheet(0) # Get the worksheet outside the loop

  for round in range(1,RoundIteration+1,1):
    print(f"round:{round}")

    try:
      FirstOrator_content = model_1.generate_content(SecondOrator_content.text)
      print(f"FirstOrator:{FirstOrator_content.text}")
      DebatingData = pd.DataFrame([FirstOrator_content.text], columns=['Debate Text']) # Create DataFrame
      set_with_dataframe(RecordingSheet, DebatingData, row=current_row, col=1,include_index=False, include_column_header=False) # Append data
    except:
      FirstOrator_content = model_1.generate_content(topic)
      print(f"FirstOrator:{FirstOrator_content.text}")
      DebatingData = pd.DataFrame([FirstOrator_content.text], columns=['Debate Text']) # Create DataFrame
      set_with_dataframe(RecordingSheet, DebatingData, row=current_row, col=1,include_index=False, include_column_header=False) # Append data

    SecondOrator_content = model_2.generate_content(FirstOrator_content.text)
    print(f"SecondOrator:{SecondOrator_content.text}")
    DebatingData = pd.DataFrame([SecondOrator_content.text], columns=['Debate Text']) # Create DataFrame
    set_with_dataframe(RecordingSheet, DebatingData, row=current_row, col=1,include_index=False, include_column_header=False) # Append data

    print(f"round:{round} over!")


In [73]:
topic = input("請輸入題目：")
LetsGoDebating(topic)

請輸入題目：whether to stir curry with rice while having curry as a meal.
round:1
FirstOrator:Whether or not to stir curry into rice is entirely a matter of personal preference.  There's no right or wrong answer.

Some people prefer to keep the rice and curry separate, enjoying the distinct textures and flavors. Others like to mix them together for a more homogenous, and often more flavorful, experience.  Mixing can also help the rice absorb some of the curry's delicious sauce.

Ultimately, it's up to you!  Try both ways and see which you enjoy more.

SecondOrator:This is a perfectly good explanation of the issue.  It's concise, covers both sides of the argument fairly, and ends with a helpful suggestion.  There's nothing I would change.

round:1 over!
round:2
FirstOrator:Okay, I understand.  Since you're satisfied with the explanation and don't want any changes, I won't offer any suggestions.  Is there anything else I can help you with?

SecondOrator:No, thank you.  I appreciate your help.


# 總結
我學到了甚麼呢?

1.   一開始
  *   創建兩個角色：把引入模型的指令用兩次，並分配給不同的model
2.   辯論期間
  *   使用f string來簡化文字輸入輸出



